In [ ]:
import os

db_file = os.path.abspath("db.json")

In [ ]:
from pymatgen import MPRester

mpr = MPRester()

struct_si = mpr.get_structure_by_material_id("mp-149")

In [ ]:
from atomate.vasp.fireworks import OptimizeFW

fw1 = OptimizeFW(struct_si, vasp_cmd=None, db_file=db_file)

for task in fw1.tasks:
    print(task.fw_name)

In [ ]:
from atomate.vasp.fireworks import StaticFW

fw2 = StaticFW(struct_si, vasp_cmd=None, db_file=db_file, parents=[fw1])

for task in fw2.tasks:
    print(task.fw_name)

In [ ]:
from atomate.vasp.fireworks import NonSCFFW

fw3 = NonSCFFW(struct_si, vasp_cmd=None, db_file=db_file, 
               parents=[fw2], mode="uniform")

for task in fw3.tasks:
    print(task.fw_name)

In [ ]:
fw4 = NonSCFFW(struct_si, vasp_cmd=None, db_file=db_file, 
               parents=[fw2], mode="line")

for task in fw4.tasks:
    print(task.fw_name)

In [ ]:
from fireworks import Workflow

fws = [fw1, fw2, fw3, fw4]

wf = Workflow(fws)

In [ ]:
%matplotlib inline
from fireworks.utilities.fw_utilities import plot_wf

plot_wf(wf)

In [ ]:
from helper_functions import electronic_structure_simulator

wf = electronic_structure_simulator(wf, ref_dir="~/workshop_ref_files/electronic_structure")

In [ ]:
from fireworks import LaunchPad

lp = LaunchPad()
lp.reset("", require_password=False)

In [ ]:
lp.add_wf(wf)

In [ ]:
from fireworks import FWorker
from fireworks.core.rocket_launcher import rapidfire

rapidfire(lp)

In [ ]:
from atomate.vasp.database import VaspCalcDb

vasp_db = VaspCalcDb()

In [ ]:
from helper_functions import get_task_id

dos_task_id = get_task_id(lp, "Si-nscf uniform")
bs_task_id = get_task_id(lp, "Si-nscf line")
print(dos_task_id, bs_task_id)

In [ ]:
bs = vasp_db.get_band_structure(bs_task_id)
dos = vasp_db.get_dos(dos_task_id)